<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Вывод:" data-toc-modified-id="Вывод:-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Вывод:</a></span></li></ul></li><li><span><a href="#Исследование-задачи" data-toc-modified-id="Исследование-задачи-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Исследование задачи</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Борьба-с-дисбалансом" data-toc-modified-id="Борьба-с-дисбалансом-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Борьба с дисбалансом</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Тестирование-модели" data-toc-modified-id="Тестирование-модели-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Тестирование модели</a></span><ul class="toc-item"><li><span><a href="#Вывод" data-toc-modified-id="Вывод-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Вывод</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Выводы</a></span></li></ul></div>

# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

## Подготовка данных

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, roc_curve, confusion_matrix


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier

In [2]:
data = pd.read_csv('/datasets/Churn.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           9091 non-null   float64
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


в столбце Tenure(сколько лет человек является клиентом банка) есть пропуски

In [4]:
data.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,9091.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,4.997690,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.894723,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,2.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


В столбце Age нет аномальных значений (18-92), в других столбцах тоже не наблюдаютс аномалии

In [5]:
#проверим на дубликаты
data.duplicated().sum()

0

In [6]:
#зполним пропуски в Tenure нелевыми значениями, чтобы сильно не влияли данные на исследования
data['Tenure'] = data['Tenure'].fillna(0)

In [7]:
#проверим кол-во пропупсков
data['Tenure'].isna().sum()

0

In [8]:
# в столбце Geography могут быть дупликаты, проверим
data['Geography'].unique()

array(['France', 'Spain', 'Germany'], dtype=object)

In [9]:
#проверим уникальные значения CustomerId - уникальный идентификатор клиента
data['CustomerId'].value_counts()

15695872    1
15801062    1
15682268    1
15647453    1
15684319    1
           ..
15629677    1
15773039    1
15766896    1
15719793    1
15812607    1
Name: CustomerId, Length: 10000, dtype: int64

### Вывод: 
- RowNumber — индекс строки в данных, не несет никакой смысловой нагрузки, можно удалить
- CustomerId — уникальный идентификатор клиента, нет дубликатов, оставим его
- Surname — фамилия, удалим, т.к есть данные CustomerId
- CreditScore — кредитный рейтинг, нет пропусков, важный столбец
- Geography — страна проживания, всего 3 страны проживания, нет дупликатов, но сделаю преобразование OHE
- Gender — пол, также подойдет преобразование OHE
- Age — возраст, верный тип данных без аномальных значений
- Tenure — сколько лет человек является клиентом банка, нет аномальных значений
- Balance — баланс на счёте, нет аномальных значений
- NumOfProducts — количество продуктов банка, используемых клиентом, нет аномальных значений
- HasCrCard — наличие кредитной карты, нет аномальных значений
- IsActiveMember — активность клиента, нет аномальных значений
- EstimatedSalary — предполагаемая зарплата, нет аномальных значений


## Исследование задачи

In [10]:
#создадим новый датафрейм для машинного обучения без лишних данных
data_ml = data.drop(['RowNumber','CustomerId', 'Surname'], axis=1)
data_ml.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [11]:
#проверим размер таблицы
data_ml.shape

(10000, 11)

In [ ]:
# Избавимся от категориальных данных методом OHE и не попадем в дамми ловушку
#data_ml = pd.get_dummies(data_ml, drop_first=True)
#data_ml.head()

In [ ]:
#data_ml.shape

В новом дата фрейме количество столбцов увеличилось на 1 столбец, т.к. мы избежали дамми ловушку вместо трех столбцов в Geography получилось два стобца, а в столбце Gender 1 столбец.

Разделим на выборки: 60% - обучающая, 20% - тестовая и 20% - валидационная

In [ ]:
#выделим целевые и признаки
features = data_ml.drop('Exited', axis=1)
target = data_ml['Exited']

In [ ]:
#выделим обучающую выборку
features_train, features_validtest, target_train, target_validtest = train_test_split(features,
                                                    target,
                                                    train_size=0.6,
                                                    random_state=12345)

In [ ]:
# выделим валидационную выборку как 50% от тестовойй
features_valid, features_test, target_valid, target_test = train_test_split(features_validtest,
                                                    target_validtest,
                                                    train_size=0.5,
                                                    random_state=12345)

In [ ]:
features_train.head()

<div class="alert alert-block alert-info">
Закодировала данные после разбиения

In [ ]:
# Избавимся от категориальных данных методом OHE и не попадем в дамми ловушку
features_train = pd.get_dummies(features_train, drop_first=True)
features_valid = pd.get_dummies(features_valid, drop_first=True)
features_test = pd.get_dummies(features_test, drop_first=True)

In [ ]:
#Для масштабирования методом scaler зафиксируем численные признаки
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']
scaler = StandardScaler()
scaler.fit(features_train[numeric])

In [ ]:
#Масштабируем числ признаки обучающей выборки
features_train[numeric] = scaler.transform(features_train[numeric])
features_train.head()

In [ ]:
#Масштабируем численные признаки валидационной выборки 
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_valid.head()

In [ ]:
#Масштабируем численные признаки тестовой выборки 
features_test[numeric] = scaler.transform(features_test[numeric])
features_test.head()

Обучение модели

In [ ]:
def all_models_accuracy(features_train, target_train, features_valid, target_valid):
    model_DTC = DecisionTreeClassifier(random_state=1515)
    DTC_score = model_DTC.fit(features_train, target_train).score(features_valid, target_valid)
    
    model_RFC = RandomForestClassifier(max_depth=7, n_estimators=43, min_samples_leaf=3, random_state=1515)
    RFC_score = model_RFC.fit(features_train, target_train).score(features_valid, target_valid)
    
    model_LgR = LogisticRegression(solver = 'liblinear')
    LgR_score = model_LgR.fit(features_train, target_train).score(features_valid, target_valid)
    
    print("Точность:" "дерево решений", DTC_score, ",", "случайный лес ", RFC_score, ",", "логистческая регрессия", LgR_score)

In [ ]:
all_models_accuracy(features_train, target_train, features_valid, target_valid)

Самая высокая точность у модели случайный лест 0.856

Проверим на адекватность

In [ ]:
#Заранее напишем функцию для изучия полноты, точности и F1-меры
def rec_prec_f1(target_valid, prediction):
    print("Полнота" , recall_score(target_valid, prediction))
    print("Точность", precision_score(target_valid, prediction))
    print("F1-мера", f1_score(target_valid, prediction))

Случайный лес

In [ ]:
model_RFC = RandomForestClassifier(random_state=1515, n_estimators = 100)
model_RFC.fit(features_train, target_train)
RFC_prediction = model_RFC.predict(features_valid)
confusion_matrix(target_valid, RFC_prediction)

In [ ]:
rec_prec_f1(target_valid, RFC_prediction)

In [ ]:
#Расчет roc_auc_RFC
RFC_probabilities_one_valid = model_RFC.predict_proba(features_valid)[:, 1]

auc_roc_RFC = roc_auc_score(target_valid, RFC_probabilities_one_valid)

auc_roc_RFC

Вывод Случайный лес примерно равную пропорцию позитивных и негативных предсказаний, выдает заметно больше качественных предсказаний, но также сильно склоняется к ложно позитивным предсказаниям (FP). Точность и качество также низкое.

Дерево решений

In [ ]:
#Матрица ошибок для дерево решений
model_DTC = DecisionTreeClassifier(random_state=1515)
model_DTC.fit(features_train, target_train)
DTC_prediction = model_DTC.predict(features_valid)
confusion_matrix(target_valid, DTC_prediction)

Вывод Матрица показала, что дерево решений склонно выдавать позитивные предсказания, очень высокое количество ложных позитивных предсказания (FP).

In [ ]:
rec_prec_f1(target_valid, DTC_prediction)

In [ ]:
#Расчет roc_auc_DTC
DTC_probabilities_one_valid = model_DTC.predict_proba(features_valid)[:, 1]

auc_roc_DTC = roc_auc_score(target_valid, DTC_probabilities_one_valid)

auc_roc_DTC

Вывод Видим низкое значение F1, следовательно низкое качество модели, проблема в точности.

Логистическая регрессия

In [ ]:
#Матрица ошибок для логистической регрессии
model_LgR = LogisticRegression(solver = 'liblinear')
model_LgR.fit(features_train, target_train)
LgR_prediction = model_LgR.predict(features_valid)
confusion_matrix(target_valid, LgR_prediction)

In [ ]:
rec_prec_f1(target_valid, LgR_prediction)

In [ ]:
#Для логистической регрессии расчитаем метрику AUC-ROC
LgR_probabilities_one_valid = model_LgR.predict_proba(features_valid)[:, 1]

auc_roc_LgR = roc_auc_score(target_valid, LgR_probabilities_one_valid)

auc_roc_LgR

In [ ]:
#исследуем баланс классов выборки
target_train.value_counts(normalize = 1)

In [ ]:
# Нарисуем гистограмму значений оттока и оставшихся клиентов
data_ml['Exited'].value_counts().plot(kind='bar',grid=True,figsize=(10,6),title='Соотношение тех, кто остался и тех кто ушел')

### Вывод
Видно, что в выборке отрицательны ответов около 80% , положитительных примерно 20%, наблюдаем дисбаланс.

## Борьба с дисбалансом

Необходмо увеличить количество положительных ответов в 4 раза для достижения баланса, или уменьшить кол-во отрицтаельных ответов, для этого:
1. Рразделим обучающую выборку на отрицательные и положительные объекты;
2. Скопируем несколько раз положительные объекты;
3. С учётом полученных данных создадим новую обучающую выборку;
4. Перемешаем данные: идущие друг за другом одинаковые вопросы не помогут обучению.
5. Обучим новые модели

In [ ]:
#Создадим функцию для увеличения представленной класса в выборке 
def upsample(features, target, repeat, upsampled_сlass):
    """Функция принимаем значение признаков (features[]), целевого признака (target[]), repeat(int / float),
    класс который будет увеличен (upsampled_сlass (0 or 1))"""
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]
    
    if upsampled_сlass == 0:
        features_upsampled = pd.concat([features_zeros]* repeat + [features_ones] )
        target_upsampled = pd.concat([target_zeros]* repeat + [target_ones] )
        features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
        
    elif upsampled_сlass == 1:
        features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
        target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
        features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    else:
        features_upsampled = 0
        target_upsampled = 0  
        
        
       
    return features_upsampled, target_upsampled

In [ ]:
#Создадим функцию для уменьшения представленной класса в выборке 
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_downsampled = pd.concat(
        [features_zeros.sample(frac=fraction, random_state=12345)] + [features_ones])
    target_downsampled = pd.concat(
        [target_zeros.sample(frac=fraction, random_state=12345)] + [target_ones])
    
    features_downsampled, target_downsampled = shuffle(
        features_downsampled, target_downsampled, random_state=12345)
    
    return features_downsampled, target_downsampled

In [ ]:
features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, fraction=0.25)

In [ ]:
#Протестируем функцию (верное значение)
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4, 0)
print(target_train_upsampled.value_counts(normalize = 1))
print(target_train_upsampled.shape)

In [ ]:
#Протестируем функцию (верное значение)
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4, 3)
features_train_upsampled

In [ ]:
#Применим функцию upsample 
#увеличим количество положительных ответов в 4 раза
features_train_upsampled, target_train_upsampled = upsample(features_train, target_train, 4, 1)
print(target_train_upsampled.value_counts(normalize = 1))
print(target_train_upsampled.shape)

In [ ]:
#Применим функцию downsample
#Уменьшим кол-в пооложительных ответов в 4 раза
#Протестируем функцию (верное значение)
features_downsampled_train, target_downsampled_train = downsample(features_train, target_train, 0.2)
print(target_downsampled_train.value_counts(normalize = 0))
print(target_downsampled_train.shape)

In [ ]:
#лучше всего выглядит upsample с ним и будем работать
target_train_upsampled.plot(kind ='hist', bins=3, figsize=(2,2))

Обучение моделей на сбалансированной выборке

In [ ]:
#Точность моделей на выборке с дисбалансом
all_models_accuracy(features_train, target_train, features_valid, target_valid)

In [ ]:
#Точность моделей на сбалансированной выборке
all_models_accuracy(features_train_upsampled, target_train_upsampled, features_valid, target_valid)

Точность на сбалансированной выборке упала.

Проверим качество ответов.

In [ ]:
#Решающее дерево
model_DTC_upsampled = DecisionTreeClassifier(random_state=1515)
model_DTC_upsampled.fit(features_train_upsampled, target_train_upsampled)
DTC_prediction_upsampled = model_DTC_upsampled.predict(features_valid)
rec_prec_f1(target_valid, DTC_prediction_upsampled)

In [ ]:
#Расчитаем auc_roc для DTC на сбалансированной выборке
DTC_upsampled_valid = model_DTC_upsampled.predict_proba(features_valid)[:, 1]

auc_roc_DTC = roc_auc_score(target_valid, DTC_upsampled_valid)

auc_roc_DTC

In [ ]:
#Случайный лес
model_RFC_upsampled = RandomForestClassifier(max_depth=7, n_estimators=43, min_samples_leaf=3, random_state=1515)
model_RFC_upsampled.fit(features_train_upsampled, target_train_upsampled)
RFC_prediction_upsampled = model_RFC_upsampled.predict(features_valid)
rec_prec_f1(target_valid, RFC_prediction_upsampled)

In [ ]:
#Расчитаем auc_roc для RFC на сбалансированной выборке
RFC_upsampled_valid = model_RFC_upsampled.predict_proba(features_valid)[:, 1]

auc_roc_RFC = roc_auc_score(target_valid, RFC_upsampled_valid)

auc_roc_RFC

In [ ]:
#Логистическая регрессия
model_LgR_upsampled = LogisticRegression(solver = 'liblinear', class_weight='balanced')
model_LgR_upsampled.fit(features_train_upsampled, target_train_upsampled)
LgR_prediction_upsampled = model_LgR_upsampled.predict(features_valid)
rec_prec_f1(target_valid, LgR_prediction_upsampled)

In [ ]:
#Расчитаем auc_roc для LgR на сбалансированной выборке
LgR_upsampled_valid = model_LgR_upsampled.predict_proba(features_valid)[:, 1]

auc_roc_LgR = roc_auc_score(target_valid, LgR_upsampled_valid)

auc_roc_LgR

### Вывод

Показаели всех моделей улучшились.

Лучшие результаты показывает алгоритм случайный лес (RandomForestClassifier). На валидационной выборке RandomForestClassifier уже показывает резульаты F1 меры = 0.61, что выше целевого целевого значения.

## Тестирование модели

In [ ]:
#Обучим финальную модель
model_RFC_final = RandomForestClassifier(
    bootstrap = True, class_weight = 'balanced', max_depth= 7,  n_estimators = 43, random_state=12345)
model_RFC_final.fit(features_train_upsampled, target_train_upsampled)

In [ ]:
model_RFC_final_prediction = model_RFC_final.predict(features_test)
rec_prec_f1(target_test, model_RFC_final_prediction)

In [ ]:
#Расчитаем auc_roc для RFC на тестовой выборке
model_RFC_final_valid = model_RFC_final.predict_proba(features_test)[:, 1]

auc_roc_RFC = roc_auc_score(target_test, model_RFC_final_valid)

auc_roc_RFC

In [ ]:
#Создаем константную модель
target_predict_constant = pd.Series([0]*len(target_test))
target_predict_constant.value_counts()

In [ ]:
#Сравним показатель точности (accuracy_score) константной модели и финальной
print('accuracy_score константой модели:', accuracy_score(target_valid, target_predict_constant))
print('accuracy_score финальной модели:', accuracy_score(target_test, model_RFC_final_prediction))
#Дополнительно сравним AUC-ROC — единственный параметр подающийся сравнению, потому что константная подель содержит только негативные ответы
print('AUC-ROC константой модели:', roc_auc_score(target_valid, target_predict_constant))
print('AUC-ROC финальной модели:', roc_auc_score(target_test, model_RFC_final_valid))

### Вывод
Финальная модель показывает результаты лучше, чем константная модель — модель можно считать адекватной.

## Выводы
В первоначальные данных наблюдался значительный дисбаланс (80% ответов целевого признака были негативными и только 20% позитивными), из-за чего обученная на этих данных модель не проходила проверку на адекватность. 

После устранения дисбаланса классов в обучающей выборки методом upsampling — увеличилось количество значений позитивного класса в 4 раза. Так мы достигли баланса классос в обучеющей выборки: 0 - 0.501043 1 - 0.498957

Разобрали несколько вариантов борьбы с дисбалансом upsampling и downsampling

На новых данных все модели показали результат выше, чем на несбалансированной выборке. Лучшие показатели были у модели случайного леса:

Полнота 0.722488038277512
Точность 0.5392857142857143
F1-мера 0.6175869120654396
AUC-ROC 0.8527604207622839
Финальная модель прошла проверку на адекватность в сравнении с контантной моделью: accuracy_score константой модели: 0.796

aaccuracy_score константой модели: 0.791
accuracy_score финальной модели: 0.7975
AUC-ROC константой модели: 0.5
AUC-ROC финальной модели: 0.8493323799115837